Допустим, у нас есть $n$ товаров с заданными стоимостями $v_i$ и массой $w_i$. В сумку убирается $С$ кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [12]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

Сформулируем задачу:
$$\max\sum v_i x_i$$
$$\sum w_i x_i \leq C $$

Как должна выглядеть задача:
$$\min c^T x$$
$$A x \leq b $$

Получается, что $c=-v$, $A=w^T$, $b=(C)$

In [1]:
import numpy as np

In [14]:
c = - np.array(values)
A = np.array(weights)         #shape = (6,)
A = np.expand_dims(A, 0)      #shape = (1,6)
b = np.array([C])

In [2]:
from scipy.optimize import linprog

In [16]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003077
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.2495783e-11])
  status: 0
 success: True
       x: array([6.18738532e-14, 1.05853306e-12, 1.21475943e-13, 7.50000000e+00,
       4.00246692e-13, 4.71394162e-13])

In [3]:
!pip install cvxpy

In [4]:
import cvxpy

In [19]:
x = cvxpy.Variable(shape=n, integer = True)

In [20]:
constraint = (A @ x <= b)
total_value = c * x

/Users/ruslansafarov/opt/anaconda3/lib/python3.8/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [10]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

/Users/ruslansafarov/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
problem.solve()

SolverError: 

                    You need a mixed-integer solver for this model. Refer to the documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    for discussion on this topic.

                    Quick fix 1: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer solver `GLPK`.

                    Quick fix 2: you can explicitly specify solver='ECOS_BB'. This may result
                    in incorrect solutions and is not recommended.
                

In [18]:
x.value

array([ 8388608.00000001, -8388608.00000001, -8388608.00000001,
        8388608.00000001,  6990509.00000001,  8388608.00000001])

Теперь положительные $x$

In [19]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()

-49.000000015906025

In [20]:
x.value

array([7.01265807e-10, 7.99333027e-10, 3.58703130e-10, 7.00000000e+00,
       4.67143021e-10, 9.34955115e-10])

Теперь $x = 0$ или $1$

In [21]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()

-17.00000000382157

In [22]:
x.value

array([1.00000000e+00, 2.25474413e-10, 2.07396078e-10, 1.00000000e+00,
       2.24003299e-10, 1.00000000e+00])

In [8]:
c = np.array([[2, 5, 3],[7, 7, 6]])
y = cvxpy.Variable(shape=(2,3), integer=True)
constraint = [ cvxpy.sum(y[0]) <= 180, 
                cvxpy.sum(y[1]) <= 220, 
                cvxpy.sum(y[:, 0]) == 110, 
                cvxpy.sum(y[:, 1]) == 150, 
                cvxpy.sum(y[:, 2]) == 140, 
                y >= 0 ]
total_value = cvxpy.sum(cvxpy.multiply(y, c))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraint)
problem.solve(solver='ECOS_BB')

1900.0000000102355

In [14]:
c = np.array([[1000, 12, 10, 19, 8], 
                   [12, 1000, 3, 7, 2], 
                   [10, 3, 1000, 6, 20], 
                   [19, 7, 6, 1000, 4], 
                   [8, 2, 20, 4, 1000]])

x = cvxpy.Variable(shape=(5,5), boolean=True)

constraint = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5) ]

total_value = cvxpy.sum(cvxpy.multiply(c, x))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraint)
problem.solve(solver='ECOS_BB')

31.999999999961364

In [16]:
import cvxpy as cvx

x = cvx.Variable(shape=(5,5), boolean=True)
u = cvx.Variable(shape=5, integer=True)
from itertools import product
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5),
    u >= 0,
    u <= 4,
    cvx.sum(cvx.diag(x)) == 0
]
for i, j in product(range(5), range(5)):
    if i >= 0 and j >= 1 and i != j:
        constraints.append(u[i] - u[j] + 5 * x[i,j] <= 4)
d = np.array([[0, 12, 10, 19, 8], 
              [12, 0, 3, 7, 2], 
              [10, 3, 0, 6, 20], 
              [19, 7, 6, 0, 4], 
              [8, 2, 20, 4, 0]])
func = cvx.sum(cvx.multiply(x, d))
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)
result = problem.solve(solver='ECOS_BB')
print(np.round(result))

32.0


In [1]:
from sympy import *
x,y,z = symbols('x y z')
f =  2*x**2-4*x*z+4*y**2-8*y*z+9*z**2+4*x+8*y-20*z
fx = f.diff(x)
print('df/dx =',fx,'= 0')
fy = f.diff(y)
print('df/dy =',fy,'= 0')
fz = f.diff(z)
print('df/dz =',fz,'= 0')
import numpy as np
np.array([0,0,0])-np.array([4,8,-20])*0.25

df/dx = 4*x - 4*z + 4 = 0
df/dy = 8*y - 8*z + 8 = 0
df/dz = -4*x - 8*y + 18*z - 20 = 0


array([-1., -2.,  5.])